In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
def get_data(batch_size=128, dims=(4, 3800)):
    return np.random.randn(batch_size, *dims), np.random.randint(0, 5, size=batch_size)

def floats_feature(arr):
    return tf.train.Feature(float_list=tf.train.FloatList(value=arr.flatten().tolist()))

def int_feature(i):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[i]))

# Write random data to tf records file

In [ ]:
features, target = get_data()
output_fname='test.tfrecords'

writer = tf.python_io .TFRecordWriter(output_fname)
for i, (np_features, np_target) in enumerate(zip(features, target)):
    tf_features_map = {
        'ch%i'%ch: floats_feature(x)
        for ch, x in enumerate(np_features)
    }
    tf_features_map['target'] = int_feature(np_target)
    tf_features = tf.train.Features(feature=tf_features_map)
    tf_example = tf.train.Example(features=tf_features)

    writer.write(tf_example.SerializeToString())

writer.close()

# Read the data from the tf records file, again

In [ ]:
f = './test.tfrecords'

feature_map = {
    'ch%i'%c: tf.FixedLenSequenceFeature(shape=(), dtype=tf.float32, allow_missing=True)
    for c in range(4)
}
feature_map['target'] = tf.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True)
fname_q = tf.train.string_input_producer([f], num_epochs=1, shuffle=False)
_, serialized = tf.TFRecordReader().read_up_to(fname_q, 128)
example = tf.parse_example(serialized, features=feature_map)
target = example['target']
target = tf.squeeze(target, axis=1)

Read all

In [ ]:
with tf.Session() as S:
    S.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    while True:
        try:
            t = S.run(example)
            print(len(t), t)
        except tf.errors.OutOfRangeError:
            print("end of file reached")
            break
    coord.request_stop()
    coord.join(threads)

Read the target data

In [ ]:
with tf.Session() as S:
    S.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    while True:
        try:
            t = S.run(target)
            print(len(t), t)
        except tf.errors.OutOfRangeError:
            print("end of file reached")
            break
    coord.request_stop()
    coord.join(threads)

Read the feature data

In [ ]:
with tf.Session() as S:
    S.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    while True:
        try:
            t = S.run(example['ch0'])
            print(len(t), t)
        except tf.errors.OutOfRangeError:
            print("end of file reached")
            break
    coord.request_stop()
    coord.join(threads)